# Weather Forecasting algorithms
### (Or how to use Linear Regression)

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model as skl
from regression import Regressors as reg

# Creating the datasets

In [2]:
# Creating a dataframe from the CSV file
data = pd.read_csv("weather_data.csv")
data

,Date,Maximum Temperature (°C),Minimum Temperature (°C),Mean Temperature (°C),Sunshine Duration (min),Shortwave Radiation (MJ/m²),Precipitation (mm),Maximum Relative Humidity (%),Minimum Relative Humidity (%),Mean Relative Humidity (%),Maximum Sea Level Pressure (hPa),Minimum Sea Level Pressure (hPa),Mean Sea Level Pressure (hPa),Maximum Wind Speed (m/s),Minimum Wind Speed (m/s),Mean Wind Speed (m/s),Wind Direction Dominant (°)
0,02/01/2008,2.610529,-4.039472,-0.716971,220.386890,5.264172,0.000000,96,59,73.500000,1022.9,1012.3,1018.27094,5.536244,0.400000,3.426518,110.169304
1,03/01/2008,5.630528,-1.179471,1.312195,149.133350,4.194036,0.000000,97,66,79.291664,1012.2,1004.9,1007.71246,6.673080,0.900000,4.628796,111.911970
2,04/01/2008,9.200529,-1.619471,3.012196,132.000000,4.972608,0.000000,96,60,80.000000,1014.2,1007.6,1010.79987,5.292448,1.500000,3.371137,114.188460
3,05/01/2008,10.950529,2.600528,7.090530,0.000000,2.124252,1.500000,89,57,70.583336,1015.1,1008.6,1012.06670,6.946222,1.421267,3.831986,201.784740
4,06/01/2008,8.540529,5.860529,7.061361,134.042560,2.159496,8.800001,98,76,90.250000,1019.7,1006.1,1013.62915,7.102817,0.900000,3.537873,246.239300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5385,30/09/2022,16.190529,6.340528,11.649693,307.843630,15.103656,0.000000,93,53,73.875000,1014.0,1007.2,1010.52910,1.964688,0.316228,1.089594,283.240500
5386,01/10/2022,16.310530,8.130529,12.479695,41.554966,5.920992,0.500000,86,61,73.750000,1019.9,1013.5,1015.44165,9.411164,1.552418,4.244933,237.491330
5387,02/10/2022,19.560530,14.260529,17.118862,480.000000,13.254948,0.000000,88,62,75.708336,1022.8,1020.2,1021.20416,8.547514,1.565247,5.617112,263.698400
5388,03/10/2022,18.200530,11.390529,14.677612,440.240400,13.985458,0.000000,93,59,78.083336,1026.3,1023.3,1024.75430,6.537584,0.282843,2.786462,347.471200


*Let's represent data as an array of floats*

*As the data is ordered by date, the Date column is not important, so we get rid of it*

In [3]:
data_array = np.array(data.drop(columns="Date").values)
data_array

array([[ 2.61052850e+00, -4.03947160e+00, -7.16971460e-01, ...,
         4.00000000e-01,  3.42651750e+00,  1.10169304e+02],
       [ 5.63052850e+00, -1.17947140e+00,  1.31219540e+00, ...,
         9.00000000e-01,  4.62879600e+00,  1.11911970e+02],
       [ 9.20052900e+00, -1.61947130e+00,  3.01219560e+00, ...,
         1.50000000e+00,  3.37113700e+00,  1.14188460e+02],
       ...,
       [ 1.95605300e+01,  1.42605290e+01,  1.71188620e+01, ...,
         1.56524750e+00,  5.61711170e+00,  2.63698400e+02],
       [ 1.82005300e+01,  1.13905290e+01,  1.46776120e+01, ...,
         2.82842730e-01,  2.78646250e+00,  3.47471200e+02],
       [ 2.09105280e+01,  8.40052900e+00,  1.37413630e+01, ...,
         1.16619040e+00,  2.96270280e+00,  1.18800026e+02]])

*We'll also need each column as a separate array*

In [4]:
max_temp = np.array(data["Maximum Temperature (°C)"])
min_temp = np.array(data["Minimum Temperature (°C)"])
mean_temp = np.array(data["Mean Temperature (°C)"])

sunshine = np.array(data["Sunshine Duration (min)"])
radiation = np.array(data["Shortwave Radiation (MJ/m²)"])
precipitation = np.array(data["Precipitation (mm)"])

max_humidity = np.array(data["Maximum Relative Humidity (%)"])
min_humidity = np.array(data["Minimum Relative Humidity (%)"])
mean_humidity = np.array(data["Mean Relative Humidity (%)"])

max_pressure = np.array(data["Maximum Sea Level Pressure (hPa)"])
min_pressure = np.array(data["Minimum Relative Humidity (%)"])
mean_pressure = np.array(data["Mean Sea Level Pressure (hPa)"])

max_wind_speed = np.array(data["Maximum Wind Speed (m/s)"])
min_wind_speed = np.array(data["Minimum Wind Speed (m/s)"])
mean_wind_speed = np.array(data["Mean Wind Speed (m/s)"])
wind_direction = np.array(data["Wind Direction Dominant (°)"])

# Necessary functions to analyse the data

In [5]:
# Standardise / Destandardise data

def to_standard(dataset):
    return (dataset - min(dataset)) / (max(dataset) - min(dataset))

def to_source(standard, source):
    return standard * (max(source) - min(source)) + min(source)

In [6]:
# This function returns an array of all sequential sub-arrays of n elements from the array

def group(array, _n):
    return np.array([array[i:i + _n] for i in range(len(array) - _n)])

In [7]:
# This function splits dataset into train and test sets

def split(dataset, point=0.8):
    pivot = int(len(dataset) * point)
    return dataset[:pivot], dataset[pivot:]

In [8]:
# Error functions

def MAE(real, predicted):
    difference = abs(real - predicted)
    return sum(difference) / len(difference)

def MSE(real, predicted):
    difference = abs(real - predicted)
    return sum(difference ** 2) / (2 * len(difference))

## What we are going to do:
We are going to use 3 methods of training and predicting data:
- For each table in dataframe predict data for the next day, according to the data from previous day of the same column
- For each table in dataframe predict data for the next day, according to the data from n previous days of the same column
- For each table in dataframe predict data for the next day, according to the data from previous day of the whole dataframe

For each method we use both sklearn and regression libraries
The results are compared with each other and with the zero theory(s)

# Method 1:
In this method we are going to predict a particular weather parameter (ex. temperature) for "tomorrow",
having only information about this parameter "today"

Taking mean temperature as an example and creating datasets

In [9]:
data_train, data_test = split(mean_temp)

X_train = data_train[:-1]
y_train = data_train[1:]

X_test = data_test[:-1]
y_test = data_test[1:]

*Scikit-learn model*

In [10]:
skl_model = skl.LinearRegression()
skl_model.fit(X_train.reshape(-1, 1), y_train)
# Show the R value
skl_model.score(X_train.reshape(-1, 1), y_train)

0.9146006523841227

*Prediction*

In [11]:
prediction = skl_model.predict(X_test.reshape(-1, 1))
prediction

array([15.46849589, 15.61110053, 14.74432287, ..., 12.46984066,
       16.90488524, 14.57104975])

*Reality*

In [12]:
y_test

array([15.765531, 14.85886 , 15.000527, ..., 17.118862, 14.677612,
       13.741363])

*Error values*

In [13]:
print("Mean Absolute Error:", MAE(y_test, prediction))
print("Mean Squared Error:", MSE(y_test, prediction))

Mean Absolute Error: 1.6468385480344996
Mean Squared Error: 2.3136632848913203


*Let's do the same operation with my own regression model*

In [14]:
reg_model = reg.LinearRegressor()
reg_model.fit(X_train.reshape(-1, 1), y_train)
# Show the R value
reg_model.score(X_train, y_train)

0.9146006523841227

*Prediction*

In [15]:
prediction = skl_model.predict(X_test.reshape(-1, 1))
prediction

array([15.46849589, 15.61110053, 14.74432287, ..., 12.46984066,
       16.90488524, 14.57104975])

*Error values*

In [16]:
print("Mean Absolute Error:", MAE(y_test, prediction))
print("Mean Squared Error:", MSE(y_test, prediction))

Mean Absolute Error: 1.6468385480344996
Mean Squared Error: 2.3136632848913203


## Results
As we can see, the results of Scikit-learn model and regression library model are completely the same.
We've got that for the first method:
- Mean Absolute Error: 1.6468385480344996 K
- Mean Squared Error: 2.3136632848913203 K

***But how much sense do these results make?***

## Zero theory
*Zero theory is an assumption, that we can do to predict the data without Machine Learning,
in order to compare its results with the ML-estimator's and to see if the using of the ML is reasonable*

In this case, we can assume that the weather doesn't change very much from day to day.
So, our *zero theory* will be that the next day the temperature wil be (approximately) the same as 'today'.

In [17]:
# Estimator function of the first zero function
# (we might have different zero functions for other methods)
def zero_func1(X):
    return X

*Prediction*

In [18]:
prediction = zero_func1(X_test)
prediction

array([15.616363, 15.765531, 14.85886 , ..., 12.479695, 17.118862,
       14.677612])

*Error values*

In [19]:
print("Mean Absolute Error:", MAE(y_test, prediction))
print("Mean Squared Error:", MSE(y_test, prediction))

Mean Absolute Error: 1.6384525693221936
Mean Squared Error: 2.3680028320543958


# Conclusion
As we can see, the errors of the linear models are approximately the same as the errors of the zero-function.
That means, that there is no use in linear models as we can make predictions with just the same quality just from an assumption.
We obtained such result because the linear model had too little information to build a better estimator.
The next methods are expected to be more effective.

# Method 2:
In this method we are going to predict a particular weather parameter (ex. pressure) for "tomorrow",
having information about this parameter for the past *n* days.

In [20]:
# First, let's pick some random n
test_n = 5

In [21]:
# Now we'll create new datasets with mean pressure as example
data_train, data_test = split(mean_pressure)

X_train = group(data_train, test_n)
y_train = data_train[test_n:]

X_test = group(data_test, test_n)
y_test = data_test[test_n:]

In [22]:
# Scikit-learn model
skl_model = skl.LinearRegression()
skl_model.fit(X_train, y_train)
# Show the R value
skl_model.score(X_train, y_train)

0.7127591598412133

*Prediction*

In [23]:
prediction = skl_model.predict(X_test)
prediction

array([1019.57327338, 1019.1199901 , 1018.67113904, ..., 1015.97976355,
       1021.42841779, 1023.11275939])

*Reality*

In [24]:
y_test

array([1019.9709 , 1019.3667 , 1021.7459 , ..., 1021.20416, 1024.7543 ,
       1022.30835])

*Error values*

In [25]:
print("Mean Absolute Error:", MAE(y_test, prediction))
print("Mean Squared Error:", MSE(y_test, prediction))

Mean Absolute Error: 3.2240711822864134
Mean Squared Error: 10.188293257490312


*Now test MultivariateRegressor of regression module*

In [26]:
reg_model = reg.MultivariateRegressor(test_n)
reg_model.fit(X_train, y_train)
# Show the R value
reg_model.score(X_train, y_train)

0.7127591598412144

*Prediction*

In [27]:
prediction = reg_model.predict(X_test)
prediction

array([1019.5732734 , 1019.11999012, 1018.67113906, ..., 1015.97976357,
       1021.42841781, 1023.11275941])

*Error values*

In [28]:
print("Mean Absolute Error:", MAE(y_test, prediction))
print("Mean Squared Error:", MSE(y_test, prediction))

Mean Absolute Error: 3.224071181693679
Mean Squared Error: 10.188293255390596


***Note:*** *Don't be confused with the absolute values of errors looking just at the raw numbers.
They are not 'larger' than the ones, there were in method 1.
As you remember, in method 1 we were predicting temperature (in Celsius degrees),
whereas now we are predicting pressure (in hectoPascale). So, they cannot be compared directly.
To see how effective the model is, we will compare it to the zero theory, but before...*

# Optimising the value of n
*If you play with the value of n, and run the code again, you can see, that the prediction also changes.
That obviously means that the precision of the estimator depends on n.
Now, as we want to train an estimator with the highest precision of predictions possible, we need to optimise the value of n.*

In this example, we'll minimise the MAE function.

In [29]:
# First, let's create of function of MAE on n:
def n_to_MAE(n):
    test_X_train = group(data_train, n)
    test_y_train = data_train[n:]

    test_X_test = group(data_test, n)
    test_y_test = data_test[n:]

    model = skl.LinearRegression()
    model.fit(test_X_train, test_y_train)

    test_prediction = model.predict(test_X_test)

    return MAE(test_y_test, test_prediction)

Although there are lots of different optimisation methods,
in this particular case the easiest and the most sufficient way is just a simple enumeration

In [30]:
# using range up to 400, because 1 year is 365 days, which can be rounded up to 400
optimal_n = 1
for n in range(1, 400):
    if n_to_MAE(n) < n_to_MAE(optimal_n):
        optimal_n = n
optimal_n

147

In [31]:
# Creating new datasets split with optimal n = 147
data_train, data_test = split(mean_pressure)

X_train = group(data_train, optimal_n)
y_train = data_train[optimal_n:]

X_test = group(data_test, optimal_n)
y_test = data_test[optimal_n:]

In [32]:
# Training the model and making a prediction
model = skl.LinearRegression()
model.fit(X_train, y_train)
prediction = model.predict(X_test)

In [33]:
# Prediction
pd.array(prediction)

<FloatingArray>
[1027.0518885164952, 1021.6400124233592, 1021.8699117996645,
   1018.11410231894, 1017.5530914157034, 1021.5904110911827,
 1027.7360412680102,  1023.545329985453,  1020.432741049831,
 1015.5364251814304,
 ...
 1013.8640483944607, 1014.6258366573825, 1009.3199500825806,
 1005.6587125897677, 1003.2068192175697,  1006.386750135083,
 1013.4450530983213, 1015.9608534376454, 1022.3074444287386,
 1023.2499092152377]
Length: 931, dtype: Float64

*Error values*

In [34]:
print("Mean Absolute Error:", MAE(y_test, prediction))
print("Mean Squared Error:", MSE(y_test, prediction))

Mean Absolute Error: 3.061828450299197
Mean Squared Error: 8.797552368492019


## Results
As we can see, *Mean Absolute Error* decreased by ***5%*** and *Mean Squared Error* by ***13.7%*** after optimization, compared to initial n = 5
We've got that for the first method:
- Mean Absolute Error: 3.061828450299197 hPa
- Mean Squared Error: 8.797552368492019 hPa

## Zero theory
In this case we can assume that the dataset can be split into such segments of length n,
that the change of the given value can be approximated as linear.
Then, the daily change will be *c = (X [ i ] - X [ i - n ]) / n* and *X [ i + 1 ] = X [ i ] + c*

In [35]:
def zero_func2(X):
    return np.array([x[-1] + (x[-1] - x[0]) / len(x) for x in X])

Note, that for zero theory optimal value for n might be different, so we'll find it once more

In [36]:
def zero_MAE(n):
    test_X_test = group(data_test, n)
    test_y_test = data_test[n:]

    test_prediction = zero_func2(test_X_test)
    return MAE(test_y_test, test_prediction)

In [37]:
optimal_n = 1
for n in range(1, 400):
    if zero_MAE(n) < zero_MAE(optimal_n):
        optimal_n = n
optimal_n

139

In [38]:
# Creating new datasets split with optimal n = 139
data_test = split(mean_pressure)[1]

X_test = group(data_test, optimal_n)
y_test = data_test[optimal_n:]

In [39]:
prediction = zero_func2(X_test)

In [40]:
print("Mean Absolute Error:", MAE(y_test, prediction))
print("Mean Squared Error:", MSE(y_test, prediction))

Mean Absolute Error: 3.3721915684832364
Mean Squared Error: 10.604314636986622


# Conclusion
In our experiment we calculated that Linear Model shows *9.2%* lower Mean Absolute Error and *17%* lower Mean Squared Error.
These results are not too impressive, but having that it doesn't take a lot of time to train, that definitely worth it.
Overall Mean Absolute Error makes about 3 hPa, or *3%* of the mean value.
The results can be improved by providing larger datasets and/or using different models.

# Method 3
In this method we are going to predict a particular weather parameter (ex. humidity) for "tomorrow",
having information about all the parameters for "today".

In [41]:
# Creating datasets
data_train, data_test = split(data_array)

X_train = data_train[:-1]
X_test = data_test[:-1]

data_train, data_test = split(mean_humidity)

y_train = data_train[1:]
y_test = data_test[1:]

## Data Standardisation
In this method we use different parameters with a wide range of values, different variation and different units.
It is known that Linear Regression doesn't work well enough with such kind of data.
To handle this, we need to standardise the data.
Standardisation is a method of representing the data in such way, that it fits in range from 0 to 1,
where 0 and 1 correspond to the minimum and maximum values in the dataset, and all the variations remain in proportion.
This can be achieved by the following formula: *standardised = (original - min) / (max - min)*

In [42]:
# Converting data into standard
X_train = np.array([to_standard(i) for i in X_train])
X_test = np.array([to_standard(i) for i in X_test])
y_train = to_standard(y_train)
y_test = to_standard(y_test)

X_train

array([[0.00647555, 0.        , 0.00323534, ..., 0.00432301, 0.00727014,
        0.11121276],
       [0.00672009, 0.        , 0.00245877, ..., 0.00205202, 0.00573158,
        0.11159831],
       [0.0106515 , 0.        , 0.00455954, ..., 0.00307089, 0.00491289,
        0.11400444],
       ...,
       [0.01786508, 0.01268466, 0.01466109, ..., 0.00049432, 0.00355743,
        0.13482435],
       [0.02255498, 0.01111934, 0.01665151, ..., 0.00093929, 0.00365002,
        0.11652841],
       [0.0173302 , 0.01129951, 0.01476854, ..., 0.        , 0.00203992,
        0.3118153 ]])

As you can see, there is no variation in column 9
So, this column doesn't give us any useful information
Let's then, just get rid of it

In [43]:
X_train = np.delete(X_train, 9, 1)
X_test = np.delete(X_test, 9, 1)

Training and scoring the model

In [44]:
reg_model = reg.MultivariateRegressor(test_n)
reg_model.fit(X_train, y_train)
reg_model.score(X_train, y_train)

0.4892230346333346

R value is relatively low, which means that predictions can be less accurate

In [45]:
prediction = reg_model.predict(X_test)

In [46]:
print("Mean Absolute Error:", MAE(y_test, prediction) * (max(mean_humidity) - min(mean_humidity)))
print("Mean Squared Error:", MSE(y_test, prediction) * (max(mean_humidity) - min(mean_humidity)))

Mean Absolute Error: 5.467852629220432
Mean Squared Error: 0.38812686947544506


Doing the same operations with the Scikit-Learn model to compare

In [47]:
skl_model = skl.LinearRegression()
skl_model.fit(X_train, y_train)
skl_model.score(X_train, y_train)

0.4892230346333346

In [48]:
prediction = reg_model.predict(X_test)

In [49]:
print("Mean Absolute Error:", MAE(y_test, prediction) * (max(mean_humidity) - min(mean_humidity)))
print("Mean Squared Error:", MSE(y_test, prediction) * (max(mean_humidity) - min(mean_humidity)))

Mean Absolute Error: 5.467852629220432
Mean Squared Error: 0.38812686947544506


Now there is no surprise that results are identical

## Zero theory
Though, most probably, there are some correlations between the various parameters,
I don't see a way to make any reasonable assumption, any better than the one, we've already seen in the method 1.

You can try to figure out one, but remember, that it shouldn't be too complicated:
You don't need to train a model, where it is easier to make an assumption that is going to be as precise,
but if for making such kind of assumption you have to train yourself to become a meteorologist, it is easier to train model anyway.

Now, as we don't have anything better, we'll use an assumption from the first method.

In [50]:
prediction = to_standard(zero_func1(split(mean_humidity)[1][:-1]))

In [51]:
print("Mean Absolute Error:", MAE(y_test, prediction) * (max(mean_humidity) - min(mean_humidity)))
print("Mean Squared Error:", MSE(y_test, prediction) * (max(mean_humidity) - min(mean_humidity)))

Mean Absolute Error: 5.9436319999999965
Mean Squared Error: 0.48390273750628265


# Results and conclusion
As a result of this experiment we created an estimator, making predictions for mean humidity with:
- Mean Absolute Error of *5.47% of humidity*, which is *8% (relatively)* lower than such for the zero theory.
- Mean Squared Error of *0.39% of humidity*, which is *19.8% (relatively)* lower than such for the zero theory.

It is barely possible to distinguish whether these results are better or worse than the ones for method 2, so we need more experiments for that.
Most probably you need to try both methods for each column and then choose which one is more precise.
Thus, and then by improving the models on larger datasets you can create a full functional weather forecast program.
(But don't forget to test models on different datasets to be sure that it is not over-fit)